# 1. 线性回归实现流程
1. 生成数据，构造输入输出数据对，并构造数据迭代器  
2. 参数初始化  
3. 定义模型结构  
4. 定义损失函数  
5. 定义优化方法  
6. 初始化超参：学习率、epoch  
7. 模型训练（注入数据，正向传播，计算loss，反向传播，计算梯度，更新参数）  


# 2. 实现流程差异之基于框架与非基于框架

## 2.1 区别一：构造迭代器
- **非基于框架**：
手动构造数据迭代器，需要对数据进行索引、打乱并按批次返回数据。

In [1]:
import random 
import torch

def data_iter(batch_size, features, labels): 
    num_examples = len(features)  # 获取训练数据的长度
    # indices = list(range(num_examples))  # 根据长度生成索引
    # random.shuffle(indices)  # 打乱索引
    indices = torch.randperm(num_examples)  # 使用 torch.randperm 生成随机索引,使用 torch.randperm() 替代 random.shuffle()，可以避免原地修改列表索引，在多线程环境中更安全。
    for i in range(0, num_examples, batch_size): 
        batch_indices = torch.tensor(indices[i: min(i + batch_size, num_examples)])  # 获取每个批次的数据索引
        yield features[batch_indices], labels[batch_indices]  # 使用 yield 生成迭代器


- **基于框架**：
通过 TensorDataset 和 DataLoader 快速构造迭代器，提供数据打乱和批量处理功能。

In [2]:
from torch.utils.data import TensorDataset, DataLoader

def load_array(data_arrays, batch_size, is_train=True, num_workers=4): 
    """构造一个 PyTorch 数据迭代器，支持多线程数据加载"""
    dataset = TensorDataset(*data_arrays)  # 打包数据
    return DataLoader(dataset, batch_size=batch_size, shuffle=is_train, num_workers=num_workers)  # 返回可迭代对象


## 2.2 区别二：定义模型结构和参数初始化
- **非基于框架**：
手动初始化模型参数，并手动实现前向传播。

In [ ]:
# 手动参数初始化
w = torch.normal(mean=0, std=0.01, size=(2, 1), requires_grad=True)  # 随机初始化权重
b = torch.zeros(1, requires_grad=True)  # 初始化偏差为 0

# 自定义线性回归模型
def linreg(X, w, b):
    return torch.matmul(X, w) + b


- **基于框架**：
使用 nn.Sequential 定义模型结构，并初始化参数。

In [ ]:
import torch.nn as nn

# 定义模型结构：线性回归
net = nn.Sequential(nn.Linear(2, 1))

# 初始化模型参数
net[0].weight.data.normal_(0, 0.01)  # 随机初始化权重
net[0].bias.data.fill_(0)  # 初始化偏差为 0


## 2.3 区别三：定义损失函数
- **非基于框架**：
手动定义平方损失函数，并确保数值稳定性。

In [ ]:
def square_loss(y_hat, y):
    """平方损失函数"""
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2


- **基于框架**：
直接使用 PyTorch 提供的 nn.MSELoss()，这是标准化的实现。

In [ ]:
loss = nn.MSELoss()

## 4.4 区别四：定义优化方法
- **非基于框架**：
手动实现随机梯度下降（SGD），并确保梯度在每次更新后清零。

In [ ]:
def sgd(params, lr, batch_size):
    with torch.no_grad():  # 禁用梯度跟踪以节省内存
        for param in params:
            param -= lr * param.grad / batch_size  # 更新参数
            param.grad.zero_()  # 将梯度清零


- **基于框架**：
直接调用库中提供的优化器。

In [ ]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

## 2.5 区别五：模型训练
模型训练包括**注入数据，正向传播，计算损失，反向传播，计算梯度，更新参数**。<br>
- **非基于框架**：


In [ ]:
lr = 0.03
num_epochs = 2
batch_size = 16
features = ''
labels = ''
for epoch in range(num_epochs):
    for X , y in data_iter(batch_size, features,labels):
        l = loss(net(X,w,b),y)
        l.sum().backward() 
        # l.backward() PyTorch的backward()方法要求计算的目标是标量，而不是张量
        sgd([w,b],lr,batch_size)
    with torch.no_grad():
        print(net(features,w,b).shape)
        train_l = loss(net(features,w,b),labels)
        print(f'epoch: {epoch + 1}, loss : {float(train_l.mean()):f}')

- **基于框架**

In [ ]:
#lr在定义优化方法的时候已经给定了
num_epoch = 3
for epoch in range(num_epoch):
    for X,y in data_iter:
        l = loss(net(X), y) #不需要输入参数
        trainer.zero_grad() #将梯度置零 A
        l.backward() #不需要sum(),已内置
        trainer.step()
				#用于更新参数 B
				#A,B这两步在非基于框架中在SGD函数中体现
    total_l =  loss(net(features),labels) 
    print(f"epoch{epoch+1}, loss{total_l:f}")